In [1]:
import torch

## Activation Functions: Sigmoid, Linear

In [2]:
class Activation_Sigmoid:
    def forward(self, inputs):
        self.output = 1 / (1 + torch.exp(-inputs))

class Activation_Linear:
    def forward(self, inputs):
        self.output = inputs

## Dense Layer

In [3]:
class DenseLayer:
    def __init__(self, n_features, n_neurons, activation_function):
        self.weights = 0.01 * torch.rand((n_features, n_neurons))
        self.biases = torch.zeros((1, n_neurons))
        self.output = None
        self.activation_function = activation_function

    def forward(self, inputs):
        weighted_sum = torch.matmul(inputs, self.weights) + self.biases
        self.activation_function.forward(weighted_sum)
        self.output = self.activation_function.output

    def get_output(self):
        return self.output

## Loss: Mean Squared Error

In [4]:
class Loss_MSE:
    def forward(self, y_pred, y_true):
        self.output = torch.mean((y_pred - y_true)**2)
        return self.output

# Neural Network Architecture

## Input Layer
- Features: 2

## Hidden Layer
- Neurons: 4
- Activation Function: Sigmoid

## Output Layer
- Neurons: 2
- Activation Function: Linear

## Training Details - Hyperparameters
- Loss Function: Mean Squared Error (MSE)
- Learning Rate: 0.001
- Number of Epochs: 1000


## Backward Propagation formula

In [13]:
def back_prop(X, y, hidden_layer, output_layer, activation_hidden, activation_output, loss_function, lr=0.01):
    # Calculate loss
    loss = loss_function.forward(output_layer.get_output(), y)

    # Backward pass for the output layer
    back_output = 2 * (output_layer.get_output() - y)  # Derivative of MSE
    output_layer.weights -= lr * torch.matmul(hidden_layer.get_output().T, back_output)
    output_layer.biases -= lr * torch.sum(back_output, axis=0, keepdim=True)

    # Backward pass for the hidden layer
    back_hidden = torch.matmul(back_output, output_layer.weights.T)
    back_hidden *= (hidden_layer.get_output() * (1 - hidden_layer.get_output()))  # Derivative of Sigmoid
    hidden_layer.weights -= lr * torch.matmul(X.T, back_hidden)
    hidden_layer.biases -= lr * torch.sum(back_hidden, axis=0, keepdim=True)

    return loss

In [16]:
# Example usage with training loop
X = torch.rand((1, 2))  # 1 sample with 2 input features
y = torch.rand((1, 2))  # Example target output for regression

activation_hidden = Activation_Sigmoid()
activation_output = Activation_Linear()  # Linear activation for regression

hidden_layer = DenseLayer(n_features=2, n_neurons=4, activation_function=activation_hidden)
output_layer = DenseLayer(n_features=4, n_neurons=2, activation_function=activation_output)

loss_function = Loss_MSE()

learning_rate = 0.001
num_epochs = 1000

# Forward pass
hidden_layer.forward(X)
output_layer.forward(hidden_layer.output)
y_pred = output_layer.output
initial_loss = loss_function.forward(y_pred, y)

print("Initial loss:", initial_loss.item())
print("Initial prediction:", y_pred)

# Training loop
for epoch in range(num_epochs):
    # Backward pass
    loss = back_prop(X, y, hidden_layer, output_layer, activation_hidden, activation_output, loss_function, lr=learning_rate)

    # Forward pass
    hidden_layer.forward(X)
    output_layer.forward(hidden_layer.get_output())
    y_pred = output_layer.output
    # Calculate error for convergence check
    err = loss_function.forward(y_pred, y)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {err.item()}")

    # Convergence check
    if err <= 0.0001:
        print("Converged at epoch", epoch)
        break

print("Final loss:", err.item())
print("Final prediction:", y_pred)
print("Target value:", y)


Initial loss: 0.15341319143772125
Initial prediction: tensor([[0.0150, 0.0158]])
Epoch 0, Loss: 0.15218563377857208
Epoch 100, Loss: 0.06812471151351929
Epoch 200, Loss: 0.03046007826924324
Epoch 300, Loss: 0.013602484948933125
Epoch 400, Loss: 0.006068127229809761
Epoch 500, Loss: 0.0027048911433666945
Epoch 600, Loss: 0.0012050358345732093
Epoch 700, Loss: 0.0005366324330680072
Epoch 800, Loss: 0.00023890910961199552
Epoch 900, Loss: 0.0001063439340214245
Converged at epoch 908
Final loss: 9.967528603738174e-05
Final prediction: tensor([[0.4466, 0.3399]])
Target value: tensor([[0.4579, 0.3484]])
